# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing Turi Create functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in Turi Create functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Fire up modules

In [1]:
import pandas as pd
import numpy as np

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 
              'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 
              'sqft_lot15':float, 'sqft_living':float, 'floors':str, 'condition':int, 'lat':float, 
              'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}
sales = pd.read_csv('kc_house_data.csv', dtype = dtype_dict)

In [3]:
sales.dtypes
sales

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180.0,5650,1,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340.0,5650.0
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570.0,7242,2,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690.0,7639.0
2,5631500400,20150225T000000,180000.0,2.0,1.00,770.0,10000,1,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720.0,8062.0
3,2487200875,20141209T000000,604000.0,4.0,3.00,1960.0,5000,1,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360.0,5000.0
4,1954400510,20150218T000000,510000.0,3.0,2.00,1680.0,8080,1,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800.0,7503.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,0263000018,20140521T000000,360000.0,3.0,2.50,1530.0,1131,3,0,0,...,8,1530,0,2009,0,98103,47.6993,-122.346,1530.0,1509.0
21609,6600060120,20150223T000000,400000.0,4.0,2.50,2310.0,5813,2,0,0,...,8,2310,0,2014,0,98146,47.5107,-122.362,1830.0,7200.0
21610,1523300141,20140623T000000,402101.0,2.0,0.75,1020.0,1350,2,0,0,...,7,1020,0,2009,0,98144,47.5944,-122.299,1020.0,2007.0
21611,0291310100,20150116T000000,400000.0,3.0,2.50,1600.0,2388,2,0,0,...,8,1600,0,2004,0,98027,47.5345,-122.069,1410.0,1287.0


# Split into training and test data Use this command to set the same seed for everyone: e.g. in python with SFrames:

In [4]:
train_data = pd.read_csv('kc_house_train_data.csv', dtype = dtype_dict)
test_data = pd.read_csv('kc_house_test_data.csv', dtype = dtype_dict)

In [5]:
train_data

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180.0,5650,1,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340.0,5650.0
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570.0,7242,2,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690.0,7639.0
2,5631500400,20150225T000000,180000.0,2.0,1.00,770.0,10000,1,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720.0,8062.0
3,2487200875,20141209T000000,604000.0,4.0,3.00,1960.0,5000,1,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360.0,5000.0
4,1954400510,20150218T000000,510000.0,3.0,2.00,1680.0,8080,1,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800.0,7503.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17379,7936000429,20150326T000000,1007500.0,4.0,3.50,3510.0,7200,2,0,0,...,9,2600,910,2009,0,98136,47.5537,-122.398,2050.0,6200.0
17380,2997800021,20150219T000000,475000.0,3.0,2.50,1310.0,1294,2,0,0,...,8,1180,130,2008,0,98116,47.5773,-122.409,1330.0,1265.0
17381,0263000018,20140521T000000,360000.0,3.0,2.50,1530.0,1131,3,0,0,...,8,1530,0,2009,0,98103,47.6993,-122.346,1530.0,1509.0
17382,0291310100,20150116T000000,400000.0,3.0,2.50,1600.0,2388,2,0,0,...,8,1600,0,2004,0,98027,47.5345,-122.069,1410.0,1287.0


In [6]:
test_data

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,0114101516,20140528T000000,310000.0,3.0,1.00,1430.0,19901,1.5,0,0,...,7,1430,0,1927,0,98028,47.7558,-122.229,1780.0,12697.0
1,9297300055,20150124T000000,650000.0,4.0,3.00,2950.0,5000,2,0,3,...,9,1980,970,1979,0,98126,47.5714,-122.375,2140.0,4000.0
2,1202000200,20141103T000000,233000.0,3.0,2.00,1710.0,4697,1.5,0,0,...,6,1710,0,1941,0,98002,47.3048,-122.218,1030.0,4705.0
3,8562750320,20141110T000000,580500.0,3.0,2.50,2320.0,3980,2,0,0,...,8,2320,0,2003,0,98027,47.5391,-122.070,2580.0,3980.0
4,7589200193,20141110T000000,535000.0,3.0,1.00,1090.0,3000,1.5,0,0,...,8,1090,0,1929,0,98117,47.6889,-122.375,1570.0,5080.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4224,8672200110,20150317T000000,1088000.0,5.0,3.75,4170.0,8142,2,0,2,...,10,4170,0,2006,0,98056,47.5354,-122.181,3030.0,7980.0
4225,5087900040,20141017T000000,350000.0,4.0,2.75,2500.0,5995,2,0,0,...,8,2500,0,2008,0,98042,47.3749,-122.107,2530.0,5988.0
4226,3448900210,20141014T000000,610685.0,4.0,2.50,2520.0,6023,2,0,0,...,9,2520,0,2014,0,98056,47.5137,-122.167,2520.0,6023.0
4227,6600060120,20150223T000000,400000.0,4.0,2.50,2310.0,5813,2,0,0,...,8,2310,0,2014,0,98146,47.5107,-122.362,1830.0,7200.0


# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [7]:
from sklearn.linear_model import LinearRegression
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
lreg = LinearRegression()
example_model = lreg.fit(train_data[example_features], train_data['price'])

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/_base.py:533: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


In [11]:
print("Intercept: ", example_model.intercept_)
print("Coefficients: ", example_model.coef_)

Intercept:  87912.86581496405
Coefficients:  [   315.40669062 -65081.88711588   6942.16598637]


# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing Turi Create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [13]:
example_predictions = example_model.predict(train_data[example_features])

In [15]:
print(example_predictions[0])

271789.26537996973


# Compute RSS

In [51]:
def get_residual_sum_of_squares(model, data, features, outcome):
    # First get the predictions
    predictions = model.predict(data[features])
    # Then compute the residuals/errors
    errors = outcome - predictions
    # Then square and add them up
    RSS = sum(errors ** 2)
    return(RSS)  

In [19]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data, example_features, test_data['price'])
print(rss_example_train)

273761940583134.06


# Create some new features

In [26]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [27]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x ** 2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x ** 2)

In [29]:
train_data['bed_bath_rooms'] = train_data['bedrooms'] * train_data['bathrooms']
test_data['bed_bath_rooms'] = test_data['bedrooms'] * test_data['bathrooms']

In [31]:
train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x: log(x))
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x: log(x))

In [33]:
train_data['lat_plus_long'] = train_data['lat'] + train_data['long']
test_data['lat_plus_long'] = test_data['lat'] + test_data['long']

In [37]:
print(test_data['bedrooms_squared'].mean())
print(test_data['bed_bath_rooms'].mean())
print(test_data['log_sqft_living'].mean())
print(test_data['lat_plus_long'].mean())

12.4466777015843
7.5039016315913925
7.550274679645938
-74.65333355403168


# Learning Multiple Models

In [43]:
model1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
print(model1_features)
model2_features = model1_features + ['bed_bath_rooms']
model3_features = model2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']


In [57]:
model1 = LinearRegression().fit(train_data[model1_features], train_data['price'])
print(model1.coef_)

[ 3.12258646e+02 -5.95865332e+04  1.57067421e+04  6.58619264e+05
 -3.09374351e+05]


In [58]:
model2 = LinearRegression().fit(train_data[model2_features], train_data['price'])
print(model2.coef_)

[ 3.06610053e+02 -1.13446368e+05 -7.14613083e+04  6.54844630e+05
 -2.94298969e+05  2.55796520e+04]


In [59]:
model3 = LinearRegression().fit(train_data[model3_features], train_data['price'])
print(model3.coef_)

[ 5.29422820e+02  3.45142296e+04  6.70607813e+04  5.34085611e+05
 -4.06750711e+05 -8.57050439e+03 -6.78858667e+03 -5.61831484e+05
  1.27334900e+05]


# Getting RSS for those models on TESTING data

In [60]:
rss_model1_test = get_residual_sum_of_squares(model1, test_data, model1_features, test_data['price'])
print(rss_model1_test)

225500469795489.8


In [61]:
rss_model2_test = get_residual_sum_of_squares(model2, test_data, model2_features, test_data['price'])
print(rss_model2_test)

223377462976465.88


In [62]:
rss_model3_test = get_residual_sum_of_squares(model3, test_data, model3_features, test_data['price'])
print(rss_model3_test)

259236319207179.94


# Getting RSS for those models on TRAINING data

In [63]:
rss_model1_train = get_residual_sum_of_squares(model1, train_data, model1_features, train_data['price'])
print(rss_model1_train)

967879963049551.5


In [64]:
rss_model2_train = get_residual_sum_of_squares(model2, train_data, model2_features, train_data['price'])
print(rss_model2_train)

958419635074069.4


In [65]:
rss_model3_train = get_residual_sum_of_squares(model3, train_data, model3_features, train_data['price'])
print(rss_model3_train)

903436455050480.2
